In [ ]:
from sympy import *
init_printing()
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

Numerische Löung von Gleichungen

In [ ]:
x = Symbol('x')

In [ ]:
xn = np.linspace(0, 4*np.pi, 850)
yn = np.tan(xn)
yn[yn>35] = np.nan
yn[yn<-10] = np.nan
plt.plot(xn, yn)
plt.plot(xn, xn)
plt.grid()
plt.axis(ymin=-5, ymax=25);

In [ ]:
glg = Eq(tan(x), x)
glg

In [ ]:
solveset(glg)

In [ ]:
#solve(glg)  # NotImplememtedError

In [ ]:
x0 = 4.  # Startwert
xi = nsolve(glg, x0)
xi

Wir haben aber die Nullstelle in der Nähe der 4 gesucht.

In [ ]:
xi = nsolve(glg, [4., 6.])
xi

Der liegt nicht zwischen 4 und 6, aber andere Löser können das:

In [ ]:
# nsolve(glg, [4., 6.], solver='bisect')
# ValueError

In [ ]:
xi1 = nsolve(glg, [4., 6.], solver='bisect', verify=False)
xi1

In [ ]:
xi2 = nsolve(glg, [4., 5.], solver='anderson') # Anderson-Björk ist eine Variante der Sekantenregel
xi2

In [ ]:
tan(xi1) - xi1

In [ ]:
tan(xi2) - xi2

In [ ]:
N(3/2*pi)

In [ ]:
nsolve(glg, [4., N(3/2*pi)], solver='bisect')

In [ ]:
nsolve(glg, [4., N(2.99/2*pi)], solver='anderson', prec=100)

Numerische Lösung als Funktion

Wir suchen die in der Nähe der 4 liegende Lösung von $\tan(x)=ax$ als Funktion von $a$

In [ ]:
a = Symbol('a')

def xi(a):
    glg = Eq(tan(x), a*x)
    return nsolve(glg, [4, 1.499*np.pi], solver='anderson')

In [ ]:
an = np.linspace(1, 20)
yn = [xi(aa) for aa in an]

In [ ]:
plt.figure()
plt.plot(an, yn)
plt.plot(an, 3*np.pi/2*np.ones_like(an));

Erhöhung der Genauigkeit

In [ ]:
import mpmath
print(mpmath.mp)

In [ ]:
mpmath.mp.dps = 50

In [ ]:
N(sqrt(2))

In [ ]:
nsolve(x**2-2, 1)

In [ ]:
sn = lambdify(x, sqrt(x), 'mpmath')
sn(2)

In [ ]:
sn(2) - N(sqrt(2), 50)

Das Taylorpolynom

In [ ]:
x = Symbol('x')
f = log(1+x)
f.series(x, 0, 10)

In [ ]:
f = sqrt(x+1)/(1-x**2)
f

In [ ]:
f.series(x, 0, 10)

In [ ]:
xn = np.linspace(-.8, .8)
nrange = range(1, 6)
fs = [f.series(x, 0, n).removeO() for n in nrange]
fs

In [ ]:
fns = [lambdify(x, ff) for ff in fs]
fn = lambdify(x, f)

In [ ]:
fns[0](xn)

In [ ]:
fns[0] = np.ones_like

In [ ]:
plt.figure()
plt.plot(xn, fn(xn), label='f')
for n, ffs in zip(nrange, fns):
    plt.plot(xn, ffs(xn), label=n)
plt.legend();

Bei der Annäherung an den Pol wird die Approximation schlechter.

Das gilt selbst dann, wenn ich den Pol nicht sehen kann, weil er komplex ist.

In [ ]:
f = atan(x)

In [ ]:
nrange = [4, 20, 60]
fs = [f.series(x, 0, n).removeO() for n in nrange]
fs[1]

In [ ]:
fns = [lambdify(x, ff, 'numpy') for ff in fs]
fn = lambdify(x, f)

In [ ]:
plt.figure()
xn = np.linspace(-2, 2, 850)
plt.plot(xn, fn(xn), label='arctan')
for n, ffs in zip(nrange, fns):
    plt.plot(xn, ffs(xn), label=f"n={n}")
plt.axis(ymin=-1, ymax=2)
plt.legend();

Wie sieht der Arcustangens im komplexen aus?

In [ ]:
df = f.diff(x)
df

In [ ]:
solve(denom(df))

In [ ]:
xn = np.linspace(-1.5, 1.5, 200)
yn = np.linspace(-1.5, 1.5, 200)
X, Y = np.meshgrid(xn, yn)
Z = X + 1j*Y

In [ ]:
W = fn(Z)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
levels = np.linspace(0, np.max(abs(W)), 300)
ax.contour(X, Y, abs(W), levels=levels);

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
R = W.real
levels = np.linspace(np.min(R), np.max(R), 300)
ax.contour(X, Y, R, levels=levels);

Logarithmische Singularität im Realteil

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
Im = W.imag
levels = np.linspace(np.min(Im), np.max(Im), 300)
ax.contour(X, Y, Im, levels=levels);

Zum Vergleich

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
levels = np.linspace(-np.pi, np.pi, 300)
ax.contour(X, Y, np.log(Z).imag, levels=levels);

Falsche Zweigwahl die Ursache für viele der Fehler der CAS

Potenzfunktionen

In [ ]:
drei_halbe = Rational(3,2)
f = x**drei_halbe
f

In [ ]:
f.subs(x, -1.)

In [ ]:
#f.subs(x, -1.).real
# AttributeError:  symbolisch ist das die Funktion `re`

In [ ]:
re(f.subs(x, -1))

In [ ]:
sqrt(-1.)

In [ ]:
I**3

In [ ]:
(-1)**3

In [ ]:
sqrt(-1)

$x^a = e^{a \log x}$

Damit ist $x^a$ für alle komplexen $a$ und für $x \in \mathbb C \setminus ]-\infty,0]$ erklärt.  (Für `sympy` sogar überall.)

In [ ]:
log(-1)

In [ ]:
f = Piecewise((x**drei_halbe, x >= 0), (-(-x)**drei_halbe, x < 0))
f

In [ ]:
f.diff(x)

In [ ]:
f.subs(x, -1)

In [ ]:
H = Piecewise((1, x>=0), (0,x<0))
H

In [ ]:
I1 = Integral(f, x)
I1

In [ ]:
I1.doit()